**Problem Set #1**

Computational Astrodynamics (ΥΦΥ204)

Implemented by: **Anastasios-Faidon Retselis (AEM: 4394)**

# Exercise 1.1

Let us first study the Molniya orbit as defined by the problem statement. We are given the inclination ($i=63.4^{\circ}$) and the eccentricity ($e=0.64$). Using the pre-defined period of the orbit ($T=12\: hours=43200\:seconds$), we can derive the semi-major axis of the orbit using Kepler's third law:

$$ 
\begin{split}
T^{2} &=\frac{4 \pi^{2}}{\mu} a^{3} \\
\Rightarrow a^{3} &=\frac{T^{2}\mu}{4\pi^{2}} \\ 
\Rightarrow a^{3} &=\frac{T^{2}G(M_{earth}+M_{satellite})}{4\pi^{2}} \\
\Rightarrow a^{3} &\approx \frac{T^{2}G(M_{earth})}{4\pi^{2}}
\end{split}
$$

and using the values $G=6.6743\times10^{-11} \frac{kg m^{3}}{s^{2}kg^{2}}$ and $M_{earth}=5.977\times10^{24}kg$ we get:

$$
\begin{split}
a &= 26617396\:m \\
\Rightarrow a &= 26617.396\:km
\end{split}
$$

Considering that for $t=\tau=0$ the line of the nodes coincides with OY axis of the ICRF reference frame and that the apogee of the orbit points towards the first point of aries (point $\gamma$), we can determine that $\Omega=270^{\circ}$ and $\omega=270^{\circ}$. If we also take into account that for $t=\tau=0$ the satellite finds itself at the perigee of the orbit, we can also determine that the mean anomaly is $M_{initial}=0$. We also note that the mean anomaly is given by the following equation:

$$
\begin{split}
M &= n(t-\tau) \\
\Rightarrow Μ &= \frac{360^{\circ}}{T}(t-\tau) 
\end{split}
$$

The orbital elements of the Molniya orbit for $t=\tau=0$ can be summarized below:

|         Orbital Element         |     Value     |
|:-------------------------------:|:-------------:|
|         Semi-major axis         |  26617396 m   |
|           Eccentricity          |     $0.64$    |
|           Inclination           |     $63.4$    |
| Longitude of the ascending node | $270^{\circ}$ |
|      Argument of pericenter     | $270^{\circ}$ |
|           Mean anomaly          |       0       |

In [ ]:
import numpy as np

def cartesian_to_orbital_elements(r_vector, v_vector):
    
    return()